***1. Creación Fact_Compras***

In [4]:
import mysql.connector

def conectar_mysql():
    # Establecer conexión a MySQL
    cnx = mysql.connector.connect(
        host='localhost', 
        user='dmonroy3',  
        password='SamuelAlejo2020%',  
        database='paad_g17' 
    )
    return cnx

def crear_tabla_fact_compras(cursor):
    # Crear la tabla Fact_Compras si no existe
    create_table_query = """
    CREATE TABLE IF NOT EXISTS Fact_Compras (
        ID_Compra VARCHAR(100),
        IDFecha INT,
        ID_Proveedor INT,
        ID_Locacion INT,
        ID_Producto VARCHAR(100),
        Fecha_Entrada DATE,
        Precio DECIMAL(10,2),
        Cantidad DECIMAL(10,2),
        Unidad_Compra INT,
        Precio_Kilo DECIMAL(10,2),
        Promedio_Precio_Kilo_SIPSA DECIMAL(10,2),
        Surplus DECIMAL(10,2),
        PRIMARY KEY (ID_Compra)
    )
    """
    cursor.execute(create_table_query)

def eliminar_tabla_fact_compras(cursor):
    # Eliminar la tabla Fact_Compras si existe
    drop_table_query = "DROP TABLE IF EXISTS Fact_Compras"
    cursor.execute(drop_table_query)

def insertar_datos_fact_compras(cursor):
    # Insertar datos en la tabla Fact_Compras
    insert_query = """
    INSERT INTO Fact_Compras (ID_Compra, IDFecha, ID_Proveedor, ID_Locacion, ID_Producto, Fecha_Entrada, Precio, Cantidad, Unidad_Compra, Precio_Kilo, Promedio_Precio_Kilo_SIPSA, Surplus)
    SELECT 
        rp.id AS ID_Compra,
        fe.IDFecha,
        dp.ID_Proveedor,
        100 AS ID_Locacion,
        CASE WHEN dpr.ID_Producto IS NULL THEN 9999 ELSE dpr.ID_Producto END AS ID_Producto,
        STR_TO_DATE(rp.delivery_date, '%d/%m/%Y') AS Fecha_Entrada,
        rp.price AS Precio,
        rp.quantity AS Cantidad,
        rp.unidad_de_compra AS Unidad_Compra,
        rp.precio_por_kilo AS Precio_Kilo,
        rp.average_precio_promedio_por_kilogramo AS Promedio_Precio_Kilo_SIPSA,
        CAST(REPLACE(rp.surplus, '%', '') AS DECIMAL(10,2)) / 100.0 AS Surplus
    FROM stg_revision_precios_sipsa rp
    LEFT JOIN dim_fecha_entrada fe ON STR_TO_DATE(rp.delivery_date, '%d/%m/%Y') = fe.Fecha
    LEFT JOIN dim_proveedor dp ON TRIM(rp.supplier_id) = TRIM(dp.Proveedor)
    LEFT JOIN dim_producto dpr ON TRIM(rp.product_id) = TRIM(dpr.ID_Producto)
    """
    cursor.execute(insert_query)
    # Obtener el número de registros insertados
    num_inserts = cursor.rowcount
    return num_inserts

def main():
    # Conectar a MySQL
    cnx = conectar_mysql()
    cursor = cnx.cursor()

    try:
        # Eliminar la tabla Fact_Compras si existe
        eliminar_tabla_fact_compras(cursor)

        # Crear la tabla Fact_Compras
        crear_tabla_fact_compras(cursor)

        # Insertar datos en la tabla Fact_Compras
        num_inserts = insertar_datos_fact_compras(cursor)

        # Confirmar los cambios
        cnx.commit()
        print(f"Se han insertado {num_inserts} registros en la tabla Fact_Compras.")

    except mysql.connector.Error as err:
        print("Error al manipular la base de datos:", err)

    finally:
        # Cerrar conexión
        cursor.close()
        cnx.close()

if __name__ == "__main__":
    main()



Se han insertado 5103 registros en la tabla Fact_Compras.
